<a href="https://colab.research.google.com/github/doudi25/SwiGlu-manual-implementation/blob/main/SwiGLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [161]:
class SwiGlu(nn.Module):
  def __init__(self,in_features,hidden_size):
    super().__init__()
    self.swish = nn.Linear(in_features,hidden_size)
    self.gate  = nn.Linear(in_features,hidden_size)
    self.proj = nn.Linear(hidden_size,in_features)
  def forward(self,x):
    self.x = x
    self.x1 = self.swish(x)
    self.swish1 = F.silu(self.x1)
    self.gate1 = self.gate(x)
    self.out = self.swish1*self.gate1
    projection = self.proj(self.out)
    self.projection = projection
    return projection

In [162]:
model = SwiGlu(5,10)

In [163]:
a = torch.randn(10,5)


In [164]:
out = model(a)

In [165]:
dout = torch.ones_like(model.projection) @ model.proj.weight
dsp =  model.out.T @ torch.ones_like(model.projection)
dswish1 = dout*model.gate1
dgate1 = model.swish1*dout
dgate = model.x.T @ dgate1
dx1 = dswish1 * ((1+torch.exp(-model.x1)+model.x1*torch.exp(-model.x1))/((1+torch.exp(-model.x1))**2))
dswish = model.x.T @ dx1

In [166]:
sum = torch.sum(model.projection)
sum.backward()

In [167]:
torch.allclose(dsp.T,model.proj.weight.grad)

True

In [168]:
torch.allclose(dgate.T,model.gate.weight.grad)

True

In [169]:
torch.allclose(dswish.T,model.swish.weight.grad)

True